# **San Francisco Rental Prices Dashboard**

Here I will be compiling the visualizations from the previous analysis, the rental analysis notebook, into functions that can be used for a Panel dashboard.

## **Import Libraries**

In [128]:
# Initial imports.
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import panel as pn
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets

In [129]:
# Initialize the Panel Extensions (for Plotly).
pn.extension('plotly')

In [130]:
# Import hvplot.pandas AFTER pn.extension('plotly') to not have functional problems. 
import hvplot.pandas

In [131]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_ACCESS_TOKEN")

In [132]:
px.set_mapbox_access_token(mapbox_token)

## **Import Data**

In [133]:
# Import the CSVs to Pandas DataFrames
csv_path = Path("sfo_neighborhoods_census_data.csv")
sf_data = pd.read_csv(csv_path, index_col="year")
sf_data.head()

file_path = Path("neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)
df_neighborhood_locations.head()

sf_data_grp2 = sf_data.groupby(["neighborhood","year"]).mean()

- - -

## **Create Visualizations**

Here I basically copied the code from my rental_analysis notebook and placed everything into seperate functions for each graph. I then used each function to create a Panel pane and then used said panes to develop a dashboard. 

In [134]:
# Define Panel Visualization Functions
def housing_units_per_year():
    sf_grp=sf_data.groupby([sf_data.index,"housing_units"]).mean()
    sf_grp=sf_grp.reset_index()
    sf_grp=sf_grp.drop(columns=["sale_price_sqr_foot", "gross_rent"])
    sf_grp_indexed=sf_grp.set_index("year")
    sf_grp_in_max=sf_grp_indexed["housing_units"].max()
    sf_grp_in_min=sf_grp_indexed["housing_units"].min()
    sf_bar=px.bar(sf_grp_indexed, y="housing_units", title="Average Amount of Housing Units in San Francisco by Year (2010 - 2016)") #(ylim=(sf_grp_in_min-4000,sf_grp_in_max+4000),legend=False, title="Average Amount of Housing Units in San Francisco by Year (2010 - 2016)")
    sf_bar.update_yaxes(range=[sf_grp_in_min-4000,sf_grp_in_max+4000])
    sf_bar.update_yaxes(title="Housing Units")
    sf_bar.update_xaxes(title="Year")
    return sf_bar
    

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    sf_gross_rent_avg=sf_data.groupby([sf_data.index,"gross_rent"]).mean()
    sf_gross_rent_avg=sf_gross_rent_avg.reset_index()
    sf_gross_rent_avg=sf_gross_rent_avg.drop(columns=["sale_price_sqr_foot","housing_units"])
    sf_gross_rent_avg=sf_gross_rent_avg.set_index("year")
    gross_rent_line=px.line(sf_gross_rent_avg,title="Average Gross Rent by Year (2010 - 2016)")
    gross_rent_line.update_yaxes(title='Average Gross Rent ($)')
    gross_rent_line.update_xaxes(title='Year')
    gross_rent_line.update_layout(showlegend=False)
    return gross_rent_line
    

def average_sales_price():
    """Average Sales Price Per Year."""
    sale_price_avg=sf_data.groupby([sf_data.index]).mean()
    sale_price_avg=sale_price_avg.drop(columns=["housing_units","gross_rent"])
    sales_price_line=px.line(sale_price_avg, title="Average Sales Price per Square Foot by Year (2010 - 2016)")
    sales_price_line.update_yaxes(title="Avg. Price per Sqr. Ft. ($)")
    sales_price_line.update_xaxes(title="Year")
    sales_price_line.update_layout(showlegend=False)
    return sales_price_line
    

def neighborhood_data(Neighborhood):
    """Average Prices by Neighborhood."""
    sf_data_grp2 = sf_data.groupby(["neighborhood","year"]).mean()
    graph_data=sf_data_grp2.drop(columns=["housing_units","gross_rent"])
    for x in sf_data_grp2.index.levels[0]:
        if Neighborhood == x:
            return graph_data.loc[Neighborhood].hvplot.line().opts(xlabel="Year", ylabel="Avg. Sale Price Per Sqr. Ft. ($)", title="Interactive Yearly Analysis of Average Price Per Square Foot Per Neighborhood")
            

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    neighborhood_mean = sf_data.groupby("neighborhood").mean()
    neighborhood_mean = neighborhood_mean.sort_values("sale_price_sqr_foot",ascending=False).reset_index()
    top_10_expensive=neighborhood_mean.head(10)
    top_10_expensive_bar_data=top_10_expensive.drop(columns=["housing_units","gross_rent"]).set_index("neighborhood")
    top_10_expensive_bar = px.bar(top_10_expensive_bar_data,title="Top 10 Most Expensive Neighborhoods With Average Sale Price Per Sqr. Foot")
    top_10_expensive_bar.update_xaxes(title="Neighborhood")
    top_10_expensive_bar.update_yaxes(title="Avg. Sale Price Per Sqr. Ft. ($)")
    top_10_expensive_bar.update_layout(showlegend=False)
    return top_10_expensive_bar


def parallel_coordinates():
    """Parallel Coordinates Plot."""
    neighborhood_mean = sf_data.groupby("neighborhood").mean()
    neighborhood_mean = neighborhood_mean.sort_values("sale_price_sqr_foot",ascending=False).reset_index()
    top_10_expensive=neighborhood_mean.head(10)
    return px.parallel_coordinates(top_10_expensive, color="sale_price_sqr_foot", title="A Parallel Coordinates Analysis of the Top Ten Most Expensive Neighborhoods", labels={"sale_price_sqr_foot": "Sale Price Per Sqr. Ft. ($)", "housing_units": "Housing Units", "gross_rent": "Gross Rent ($)"})


def parallel_categories():
    """Parallel Categories Plot."""
    neighborhood_mean = sf_data.groupby("neighborhood").mean()
    neighborhood_mean = neighborhood_mean.sort_values("sale_price_sqr_foot",ascending=False).reset_index()
    top_10_expensive=neighborhood_mean.head(10)
    return px.parallel_categories(
    top_10_expensive,
    dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
    color_continuous_scale=px.colors.sequential.Inferno,
    color="sale_price_sqr_foot",
    labels={"neighborhood": "Neighborhood", "sale_price_sqr_foot": "Sale Price Per Sqr. Ft. ($)", "housing_units": "Housing Units", "gross_rent": "Gross Rent ($)"},
    title="A Parallel Categories Analysis of the Top Ten Most Expensive Neighborhoods")


def neighborhood_map():
    """Neighborhood Map"""
    neighborhood_mean = sf_data.groupby("neighborhood").mean()
    neighborhood_mean = neighborhood_mean.sort_values("sale_price_sqr_foot",ascending=False).reset_index()
    mapbox_mean=neighborhood_mean.sort_values("neighborhood").set_index("neighborhood").reset_index()
    combined_df=pd.concat([df_neighborhood_locations,mapbox_mean], axis='columns')
    combined_df=combined_df.drop(columns="neighborhood")
    combined_df=combined_df.rename(columns={"sale_price_sqr_foot": "Sale Price Per Sqr. Ft.", "housing_units": "Housing Units", "gross_rent": "Gross Rent"})
    map=px.scatter_mapbox(combined_df,lat="Lat",lon="Lon",size="Sale Price Per Sqr. Ft.",color="Gross Rent",zoom=11,title="An Interactive Map of Average Sale Price Per Sqr. Ft. Per Neighborhood");
    return map


### **Panel Dashboard**

Here I then turned the above functions into panes and designed my dashboard. 

In [135]:
housing_units_per_year_pane =pn.pane.Plotly(housing_units_per_year())

In [136]:
housing_units_per_year_pane.pprint()

Plotly(Figure)


In [137]:
average_gross_rent_pane=pn.pane.Plotly(average_gross_rent())

In [138]:
average_gross_rent_pane.pprint()

Plotly(Figure)


In [139]:
average_sales_price_pane=pn.pane.Plotly(average_sales_price())

In [140]:
average_sales_price_pane.pprint()

Plotly(Figure)


In [141]:
average_price_neighborhood_pane= interact(neighborhood_data,Neighborhood=sf_data_grp2.index.levels[0])

In [142]:
average_price_neighborhood_pane.pprint()

Column
    [0] Column
        [0] Select(name='Neighborhood', options=['Alamo Square', ...], value='Alamo Square')
    [1] Row
        [0] HoloViews(Curve, name='interactive03215')


In [143]:
top_most_expensive_nbh_pane=pn.pane.Plotly(top_most_expensive_neighborhoods())

In [144]:
housing_units_per_year_pane

Plotly(Figure)

In [145]:
average_gross_rent_pane

Plotly(Figure)

In [146]:
average_sales_price_pane

Plotly(Figure)

In [147]:
average_price_neighborhood_pane

Column
    [0] Column
        [0] Select(name='Neighborhood', options=['Alamo Square', ...], value='Alamo Square')
    [1] Row
        [0] HoloViews(Curve, name='interactive03215')

In [148]:
top_most_expensive_nbh_pane

Plotly(Figure)

In [149]:
parallel_coordinates_pane=pn.pane.Plotly(parallel_coordinates())

In [150]:
parallel_coordinates_pane

Plotly(Figure)

In [151]:
parallel_coordinates_pane.pprint()

Plotly(Figure)


In [152]:
parallel_categories_pane=pn.pane.Plotly(parallel_categories())

In [153]:
parallel_categories_pane

Plotly(Figure)

In [154]:
parallel_categories_pane.pprint()

Plotly(Figure)


In [155]:
neighborhood_map_pane=pn.pane.Plotly(neighborhood_map())

In [156]:
neighborhood_map_pane

Plotly(Figure)

In [157]:
# Create a setupe for all of the graphs analyzing average price per square foot and etc.
line_graph_row=pn.Row(average_sales_price_pane, average_gross_rent_pane)

In [158]:
line_graph_column=pn.Column(
    '# RETURNS ANALYSIS',
    '## Annual Averages from 2010-2016',
    'I consider this the "Returns Analysis" because while no initial investment has been made, one can already see all the opportunity that has been missed. Here we have an interactive line graph with the Average Price per Square Foot based on each individual neighborhood. We then also see the average gross rent. One can see the general constant uptrends, hence the returns analysis. Don\'t miss out.',
    average_price_neighborhood_pane,
    line_graph_row)

In [159]:
line_graph_column

Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Markdown(str)
    [3] Column
        [0] Column
            [0] Select(name='Neighborhood', options=['Alamo Square', ...], value='Alamo Square')
        [1] Row
            [0] HoloViews(Curve, name='interactive03215')
    [4] Row
        [0] Plotly(Figure, viewport={'xaxis.range': [2010, ...})
        [1] Plotly(Figure, viewport={'xaxis.range': [2010, ...})

In [160]:
most_expensive_row=pn.Row(parallel_coordinates_pane,parallel_categories_pane)

In [161]:
most_expensive_column=pn.Column(
    '# **THE BIG TEN**',
    '## **The Top 10 Most Expensive Neighborhoods**',
    'Here we have an analysis of the top 10 most expensive neighborhods in San Francisco between the years of 2010-2016. Here one can see the vast opportunities of investing in these areas. Union Square District in itself is almost $1000 per square foot on average.',
    top_most_expensive_nbh_pane,
    most_expensive_row)

In [162]:
most_expensive_column

Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Markdown(str)
    [3] Plotly(Figure, viewport={'xaxis.range': [-0.5, ...})
    [4] Row
        [0] Plotly(Figure)
        [1] Plotly(Figure)

In [163]:
welcome_column=pn.Column(
    '# **Welcome**',
    '## San Francisco Real Estate Analysis from 2010-2016',
    'Welcome to a visual and interactive analysis of San Franciscan Real Estate from 2010 to 2016. Between these three tabs one will find 8 different visual representations of the Real Estate market in San Francisco. To begin, below we find an interactive map of average sale price per square foot per neighborhood with color illustrating the gross rent; and a bar graph of the average amount of housing units in San Francisco by year. This illustrates the most expensive areas and also illustrates that there is opportunity to get involved in the market.',
    neighborhood_map_pane, 
    housing_units_per_year_pane)

In [164]:
welcome_column

Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Markdown(str)
    [3] Plotly(Figure, relayout_data={'mapbox.center': {'lon': ...})
    [4] Plotly(Figure, viewport={'xaxis.range': [2009.5, ...})

In [165]:
dashboard_tabs=pn.Tabs(("Welcome",welcome_column),("Returns Analysis",line_graph_column),("The Big 10",most_expensive_column))

In [166]:
dashboard_tabs

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Markdown(str)
        [3] Plotly(Figure, relayout_data={'mapbox.center': {'lon': ...})
        [4] Plotly(Figure, viewport={'xaxis.range': [2009.5, ...})
    [1] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Markdown(str)
        [3] Column
            [0] Column
                [0] Select(name='Neighborhood', options=['Alamo Square', ...], value='Alamo Square')
            [1] Row
                [0] HoloViews(Curve, name='interactive03215')
        [4] Row
            [0] Plotly(Figure, viewport={'xaxis.range': [2010, ...})
            [1] Plotly(Figure, viewport={'xaxis.range': [2010, ...})
    [2] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Markdown(str)
        [3] Plotly(Figure, viewport={'xaxis.range': [-0.5, ...})
        [4] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)

## Serve the Panel Dashboard

Here I will make my dashboard a servable object which will allow it to be launcehd as a web app. 

In [168]:
dashboard_tabs.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Markdown(str)
        [3] Plotly(Figure, relayout_data={'mapbox.center': {'lon': ...})
        [4] Plotly(Figure, viewport={'xaxis.range': [2009.5, ...})
    [1] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Markdown(str)
        [3] Column
            [0] Column
                [0] Select(name='Neighborhood', options=['Alamo Square', ...], value='Alamo Square')
            [1] Row
                [0] HoloViews(Curve, name='interactive03215')
        [4] Row
            [0] Plotly(Figure, viewport={'xaxis.range': [2010, ...})
            [1] Plotly(Figure, viewport={'xaxis.range': [2010, ...})
    [2] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Markdown(str)
        [3] Plotly(Figure, click_data={'points': [{'curveNumber'...}, viewport={'xaxis.range': [-0.5, ...})
        [4] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)